In [1]:
import os
os.environ['LC_CTYPE'] = 'en_US.UTF-8'

!pip install langchain #For numbers of task provide extract context to llm
!pip install pypdf #read pdf document and extract information
!pip install sentence_transformers #Embedding
! pip install chromadb #Store Text data and embedding
!pip install accelerate
!pip install --upgrade accelerate
%pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: jsonpatch
    Found existing installation: jsonpatch 1.32
    Uninstalling jsonpatch-1.32:
      Successfully uninstalled jsonpatch-1.32
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=32d17eafc7c51d844c6df660680c13487e0aea84280b1a04a9c8572696780d97
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 95.2 MB/

In [2]:
from langchain.docstore.document import Document
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [4]:
loader = PyPDFLoader('/kaggle/input/healthdata/HealthData.pdf')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=40)
documents = text_splitter.split_documents(documents)

In [5]:
from sentence_transformers import SentenceTransformer, models

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

Vector_db = Chroma.from_documents(collection_name="document_docs", documents=documents, embedding=hf_embed, persist_directory="/content/drive/MyDrive/Conv_bot/Data/Docs/Persist_dir")
Vector_db.similarity_search("dummy")
Vector_db.persist()
pdf_vector_db_path = "/content/drive/MyDrive/Conversational_bot/Data/Docs/db_path"

hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = Chroma(collection_name="document_docs", embedding_function=hf_embed, persist_directory=pdf_vector_db_path)

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
def get_similar_docs(question, similar_doc_count):
  return Vector_db.similarity_search(question, k=similar_doc_count)
for doc in get_similar_docs("lump under left nipple and stomach pain (male) Hi,I have recently noticed a few weeks ago a lump under my nipple, it hurts to touch and is about the size of a quarter. Also I have bern experiencing stomach pains that prevent me from eating. I immediatly feel full and have extreme pain. Please help",2):
  print(doc.page_content)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

148 
 PEPTIC ULCER DISEASE  
 
Gastric and duodenal ulcers usually cannot be differentiated based on history alone, although some 
findings may be suggestive. Epigastric pain is the most common symptom of both ga stric and duodenal 
ulcers. It is characterized by a gnawing or burning sensation and occurs after meals —classically, shortly 
after meals with gastric ulcer and 2 -3 hours afterward with duodenal ulcer.  
 
Possible manifestations include the following:  
 Dyspeps ia, including belching, bloating, distention, and fatty food intolerance  
 Heartburn  
 Chest discomfort  
 Hematemesis or melena resulting from gastrointestinal bleeding. Melena may be intermittent 
over several days or multiple episodes in a single day.  
 Rarel y, a briskly bleeding ulcer can present as hematochezia.  
 Symptoms consistent with anemia (eg, fatigue, dyspnea) may be present  
 Sudden onset of symptoms may indicate perforation.  
 NSAID -induced gastritis or ulcers may be silent, especiall

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain

In [12]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction:
  if you are a doctor please provide reasonable causes for the following symptoms.

  {context}

  Question: {question}

  Response:
  """
def build_qa_chain():
  torch.cuda.empty_cache()
  model_name = "databricks/dolly-v2-3b" # can use dolly-v2-3b or dolly-v2-7b for smaller model and faster inferences.

  # device = "cuda:0" if torch.cuda.is_available() else "cpu"
  # Increase max_new_tokens for a longer response
  # Other settings might give better results! Play around
  instruct_pipeline = pipeline(model=model_name, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto", return_full_text=True, max_new_tokens=256, top_p=0.95, top_k=50,model_kwargs={'load_in_8bit': False})
  model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)

  prompt = PromptTemplate(input_variables=['context', 'question'], template=template)

  hf_pipe = HuggingFacePipeline(pipeline=instruct_pipeline)
  return load_qa_chain(llm=hf_pipe, chain_type="stuff", prompt=prompt, verbose=True)# Note: if you use dolly 12B or smaller model but a GPU with less than 24GB RAM, use 8bit. This requires %pip install bitsandbytes
  # instruct_pipeline = pipeline(model=model_name, trust_remote_code=True, device_map="auto", model_kwargs={'load_in_8bit': True})

In [14]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [15]:
import torch
from transformers import pipeline
!pip install accelerate
!pip install -i https://test.pypi.org/simple/bitsandbytes
qa_chain = build_qa_chain()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ERROR: You must give at least one requirement to install (see "pip help install")


A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [17]:
def displayHTML(html):
    """Display HTML in Jupyter notebook."""
    from IPython.display import HTML
    display(HTML(html))

In [18]:
def answer_question(question):
  similar_docs = get_similar_docs(question, similar_doc_count=3)
  result = qa_chain({"input_documents": similar_docs, "question": question})
  result_html = f"<p><blockquote style=\"font-size:24\">{question}</blockquote></p>"
  result_html += f"<p><blockquote style=\"font-size:18px\">{result['output_text']}</blockquote></p>"
  result_html += "<p><hr/></p>"
  for d in result["input_documents"]:
    source_id = d.metadata["source"]
    result_html += f"<p><blockquote>{d.page_content}<br/>(Source: <a href=\"https://gardening.stackexchange.com/a/{source_id}\">{source_id}</a>)</blockquote></p>"
  displayHTML(result_html)

In [19]:
answer_question("I feel dizzy. Can you please tell me what are the causes ")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction:
  if you are a doctor please provide reasonable causes for the following symptoms.

  230 
 Diagnosis  
Tension -type headache, migraine (with or without aura) and cluster headache  
 Diagnose tension -type headache, migraine or cluster headache according to the headache 
features in the table  
Migraine with aura  
 Suspect aura in people who present with or without hea dache and with neurological symptoms 
that:  
o are fully reversible and 
o develop gradually, either alone or in succession, over at least 5 minutes and 
o last for 5 –60 minutes  
 Diagnose migraine with aura in people who present with or without headache and with one or 
more of the following typical aura symptoms that meet the criteria (see above):  
o visual symptoms that may be positive

In [20]:
!pip install transformers

In [21]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationSummaryBufferMemory

In [22]:
def build_qa_chain():
  torch.cuda.empty_cache()
  # Defining our prompt content.
  # langchain will load our similar documents as {context}
  template = """You are a chatbot having a conversation with a human. your job is to help providing the best answer for questions related to health.
  Use only information in the following paragraphs to answer the question at the end. Explain the answer with reference to these paragraphs.
  If you don't have relavent information in paragraph then give response "I dont't know".

  {context}

  {chat_history}

  {human_input}

  Response:
  """
  prompt = PromptTemplate(input_variables=['context', 'human_input', 'chat_history'], template=template)

  # Increase max_new_tokens for a longer response
  # Other settings might give better results! Play around
  model_name = "databricks/dolly-v2-3b" # can use dolly-v2-3b, dolly-v2-7b or dolly-v2-12b for smaller model and faster inferences.
  instruct_pipeline = pipeline(model=model_name, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto",
                               return_full_text=True, max_new_tokens=256, top_p=0.95, top_k=50, model_kwargs={'load_in_8bit': True})
  # instruct_pipeline = pipeline(model=model_name, trust_remote_code=True, device_map="auto", model_kwargs={'load_in_8bit': True})

  hf_pipe = HuggingFacePipeline(pipeline=instruct_pipeline)

  # Add a summarizer to our memory conversation
  # Let's make sure we don't summarize the discussion too much to avoid losing to much of the content

  # Models we'll use to summarize our chat history
  # We could use one of these models: https://huggingface.co/models?filter=summarization. facebook/bart-large-cnn gives great results, we'll use t5-small for memory
  summarize_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small", device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)
  summarize_tokenizer = AutoTokenizer.from_pretrained("t5-small", padding_side="left", model_max_length = 512)
  pipe_summary = pipeline("summarization", model=summarize_model, tokenizer=summarize_tokenizer) #, max_new_tokens=500, min_new_tokens=300
  # langchain pipeline doesn't support summarization yet, we added it as temp fix in the companion notebook _resources/00-init
  # hf_summary = HuggingFacePipeline_WithSummarization(pipeline=pipe_summary)
  # HuggingFacePipeline
  hf_summary = HuggingFacePipeline(pipeline=pipe_summary)

  #will keep 500 token and then ask for a summary. Removes prefix as our model isn't trained on specific chat prefix and can get confused.
  memory = ConversationSummaryBufferMemory(llm=hf_summary, memory_key="chat_history", input_key="human_input", max_token_limit=300, human_prefix = "", ai_prefix = "")

  # Set verbose=True to see the full prompt:
  print("loading chain, this can take some time...")
  return load_qa_chain(llm=hf_pipe, chain_type="stuff", prompt=prompt, verbose=True, memory=memory)

In [23]:
class ChatBot():
  def __init__(self, db):
    self.reset_context()
    self.db = db

  def reset_context(self):
    self.sources = []
    self.discussion = []
    # Building the chain will load Dolly and can take some time depending on the model size and your GPU
    self.qa_chain = build_qa_chain()
    displayHTML("<h1>Hi! I'm a chat bot Doctor. How Can I help you ?</h1>")

  def get_similar_docs(self, question, similar_doc_count):
    return self.db.similarity_search(question, k=similar_doc_count)

  def chat(self, question):
    # Keep the last 3 discussion to search similar content
    self.discussion.append(question)
    similar_docs = self.get_similar_docs(" \n".join(self.discussion[-3:]), similar_doc_count=2)
    # Remove similar doc if they're already in the last questions (as it's already in the history)
    similar_docs = [doc for doc in similar_docs if doc.metadata['source'] not in self.sources[-3:]]

    result = self.qa_chain({"input_documents": similar_docs, "human_input": question})
    # Cleanup the answer for better display:
    answer = result['output_text'].capitalize()
    result_html = f"<p><blockquote style=\"font-size:24\">{question}</blockquote></p>"
    result_html += f"<p><blockquote style=\"font-size:18px\">{answer}</blockquote></p>"
    result_html += "<p><hr/></p>"
    for d in result["input_documents"]:
      source_id = d.metadata["source"]
      self.sources.append(source_id)
      result_html += f"<p><blockquote>{d.page_content}<br/>(Source: <a href=\"https://gardening.stackexchange.com/a/{source_id}\">{source_id}</a>)</blockquote></p>"
    displayHTML(result_html)

chat_bot = ChatBot(Vector_db)

loading chain, this can take some time...


In [24]:
def displayHTML(html):
    """Display HTML in Jupyter notebook."""
    from IPython.display import HTML
    display(HTML(html))

#Initiate the bot
chat_bot.chat("I feel feverish.Can you please help me?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human. your job is to help providing the best answer for questions related to health.
  Use only information in the following paragraphs to answer the question at the end. Explain the answer with reference to these paragraphs.
  If you don't have relavent information in paragraph then give response "I dont't know".

  16 
 FEVER  
Fever is the temporary increase in the body's temperature in response to  a disease or illness.  
A child has a fever when the temperature is at or above one of these levels:  
 38°C  measured in the bott om (rectally)  
 37.5°C measured in the mouth (orally)  
 37.2°C  measured under the arm (axillary)  
An adult probably has a fever when the temperature is above 37.2 - 37.5°C , depending on the time of 
day.  
A symptom is something the patient reports and feels, whi le a sign is something other people, 

/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")



> Finished chain.

> Finished chain.


In [25]:
chat_bot.chat("I measured my temperature using a digital thermometer. It recorded 100 degree fahrenheit.")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human. your job is to help providing the best answer for questions related to health.
  Use only information in the following paragraphs to answer the question at the end. Explain the answer with reference to these paragraphs.
  If you don't have relavent information in paragraph then give response "I dont't know".

  

  : I feel feverish.Can you please help me?
: 
You describe yourself feeling feverish.  You describe your temperature as being 38 degrees celsius.  First I will need to know your history.   I will need to know the history you give of how your temperature was measured, what the temperatures were prior to presentation and what the current activity level, if any, is like now.  After I have your history I will go into the physical exam.

  I measured my temperature using a digital thermometer. It recorded 100 degree fahrenh

/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")



> Finished chain.

> Finished chain.


In [26]:
chat_bot.chat("I had soar throat,cough,running nose")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human. your job is to help providing the best answer for questions related to health.
  Use only information in the following paragraphs to answer the question at the end. Explain the answer with reference to these paragraphs.
  If you don't have relavent information in paragraph then give response "I dont't know".

  

  : I feel feverish.Can you please help me?
: 
You describe yourself feeling feverish.  You describe your temperature as being 38 degrees celsius.  First I will need to know your history.   I will need to know the history you give of how your temperature was measured, what the temperatures were prior to presentation and what the current activity level, if any, is like now.  After I have your history I will go into the physical exam.
: I measured my temperature using a digital thermometer. It recorded 100 degree fahrenhe

/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")



> Finished chain.

> Finished chain.


In [27]:
chat_bot.chat("What could be the reason for my fever?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human. your job is to help providing the best answer for questions related to health.
  Use only information in the following paragraphs to answer the question at the end. Explain the answer with reference to these paragraphs.
  If you don't have relavent information in paragraph then give response "I dont't know".

  

  : I feel feverish.Can you please help me?
: 
You describe yourself feeling feverish.  You describe your temperature as being 38 degrees celsius.  First I will need to know your history.   I will need to know the history you give of how your temperature was measured, what the temperatures were prior to presentation and what the current activity level, if any, is like now.  After I have your history I will go into the physical exam.
: I measured my temperature using a digital thermometer. It recorded 100 degree fahrenhe

/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")



> Finished chain.

> Finished chain.


In [28]:
chat_bot.chat("Can you suggest any medictaions?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human. your job is to help providing the best answer for questions related to health.
  Use only information in the following paragraphs to answer the question at the end. Explain the answer with reference to these paragraphs.
  If you don't have relavent information in paragraph then give response "I dont't know".

  

  : I feel feverish.Can you please help me?
: 
You describe yourself feeling feverish.  You describe your temperature as being 38 degrees celsius.  First I will need to know your history.   I will need to know the history you give of how your temperature was measured, what the temperatures were prior to presentation and what the current activity level, if any, is like now.  After I have your history I will go into the physical exam.
: I measured my temperature using a digital thermometer. It recorded 100 degree fahrenhe

/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")



> Finished chain.

> Finished chain.
